#### Define the Siamese Model and Train 

#### Load the Images datasets

In [1]:
# Load the libraries
import os
from siamese import Siamese
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

# Create the constants for the data folders
ANC_PATH = os.path.join('data', 'anchor')
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')

# Constants
BATCH_SIZE = 16
PRE_FETCH_SIZE = 8

#### Set GPU Growth 

In order to avoid OutOfMemory error we have to use GPU power.

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Load the data for the training

In [3]:
# Load 400 of the paths for each category (helps build a clean data pipeline)
anchor_itt   = tf.data.Dataset.list_files(ANC_PATH + '/*.jpg').take(400)
positive_itt = tf.data.Dataset.list_files(POS_PATH + '/*.jpg').take(400)
negative_itt = tf.data.Dataset.list_files(NEG_PATH + '/*.jpg').take(400)

Metal device set to: Apple M2 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



#### Data Preprocessing  

In [4]:
# Normalize the image values from [0,255] to [0,1] for a better Gradient Descend process

def normalize(image_path):
    # Load image
    byte_img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(byte_img)

    # Normalize in [0,1] and resize to 100x100x3 for the model
    img = tf.image.resize(img, (100,100))
    img = img / 255.0

    return img

#### Create labeled dataset

In [5]:
# Create a dataset that includes both positive and negative examples
positive = tf.data.Dataset.zip((anchor_itt, positive_itt, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor_itt))))) # type: ignore
negative = tf.data.Dataset.zip((anchor_itt, negative_itt, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor_itt))))) # type: ignore

dataset = tf.data.Dataset.concatenate(positive, negative)

#### Create the Train-Test datasets

In [6]:
# Create a normalize function for the dataset object that has type (path, path, label)
def data_normalize(input_img, val_image, label):
    return(normalize(input_img), normalize(val_image), label)

In [7]:
# Normalize all the data of the dataset and build the data pipeline (need to shuffle)
dataset = dataset.map(data_normalize)
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size=1024)

In [8]:
# Train-Test split
train, test = tf.keras.utils.split_dataset(dataset, left_size=0.8)

# Create the batch size to train and test the model
train = train.batch(BATCH_SIZE)
train = train.prefetch(PRE_FETCH_SIZE)

test = test.batch(BATCH_SIZE)
test = test.prefetch(PRE_FETCH_SIZE)

2023-06-08 23:38:36.955066: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


### Define the Siamese Model

In [9]:
# Create the Siamese model
siamese_model = Siamese.siamese_model()
siamese_model.summary()

Model: "SiameseModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_image (InputLayer)  [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_image[0][0]',            
                                                                  'validation_image[0][